# Interpolating and fitting data using Python


Often, data are provided in the form of plots.  While this is useful if we are interested in qualitative trends, this makes using the data in calculations awkward.  In these situations, we can use digitization programs, such as
[Web Plot Digitizer](https://apps.automeris.io/wpd/)
to convert images into tables of data, which as more convenient to utilize.

In this workbook, we will be using Python to interpolate and fit data that are provided in tabular format, such as in CSV files.  This encodes the data in a form that can be easily evaluated by the computer and subsequently used in more sophistocated manners.

First we will look at interpolating data using cubic splines.  This utilizes the routines `splrep` and `splev` from the `scipy.interpolate` library.  Then we cover fitting of mathematical models using the method of least squares.  This will use the curve fitting routine `curve_fit` from the `scipy.optimize` library.



## Interpolation of data

In some cases, the data you are provided are fairly smooth, with negligible uncertainty.  For these situations, you can use a cubic spline fit through the data to interpolate (or even extrapolate) the information to regions where you do not have any values.

To give an example, let's generate some data for a function at a random set of points.


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


# Pick a random set of points to tabulate a function
x_data = np.random.uniform(0.0, 10.0, 20)
x_data.sort()   # We need to sort the order of these points.

# These are the values of the function at the chosen points.
y_data = [np.sin(x)*np.exp(-x) for x in x_data]


# pack the data into a Pandas dataframe
df = pd.DataFrame({'x': x_data, 'y': y_data})
print(df)


# Let's plot the data
plt.plot(df['x'], df['y'], 'o')
plt.xlabel(r'$x$')
plt.ylabel(r'$y$')
plt.xlim([0.0, 10.0])
plt.show()

Now let's fit the tabulated data with a cubic spline.  To do this, we use the function `splrep` from the library `scipy.interpolate`; this function takes a list of $x$ values and a list of the $y$ values to create a spline representation of the data, which is stored in the variable `fit`.

To find the value of the function at any position $x$, we just us the function `splev`.  This takes as arguments the value of the variable $x$ at which we want to evaluate the function and the variable `fit` which contains the spline representation of the function.

In [ ]:
from scipy.interpolate import splrep, splev, splint


# Create the spline representation of the data and put it in the variable fit
fit = splrep(x_data, y_data)

# Now let's plot the spline fit
x_interp = np.linspace(0.0, 10, 100)

# Evaluate the spline fit at each value of x in the list x_interp
y_interp = [splev(x, fit) for x in x_interp]

# We want to compare the spline fit to the original set of data we were given.
y_exact = [np.sin(x)*np.exp(-x) for x in x_interp]


# Heres the plot
plt.plot(x_data, y_data, 'o')
plt.plot(x_interp, y_interp, label='cubic interpolation')
plt.plot(x_interp, y_exact, color='blue', ls='dashed', label='exact')
plt.xlabel(r'$x$')
plt.ylabel(r'$y$')
plt.xlim([0.0, 10.0])
plt.legend()
plt.show()



## Least squares fit of data

Let's suppose that based on some theory, we know that data should follow the following mathematical expression
$$
y(x) = A e^{-a x} + b
$$
where $A$, $a$, and $b$ are parameters of the model, which are unknown to us.  Given some experimental measurements, we want to determine the values of the unknown parameters by finding the best fit of the theory through the data.  

To give an explicit example, let's first generate some "fake" data with added Gaussian noise.  

In [ ]:
import numpy as np
import pylab as plt
from scipy.optimize import minimize


# choose a set of 100 uniformly spaced points from 0 to 10 at which we perform "measurements"
x_data = np.linspace(0.0, 10.0, 100)

# This is the "exact" value of the measurements, which we assume is unknown.
# Note that in this case, we have A=1, a=1, and b=0.
y_exact = [np.exp(-x) for x in x_data]

# We will add Gaussian noise to the measurements, with a mean of 0 and a standard
# deviation of sig.
sig = 0.1
y_data = [y + np.random.normal(0.0, sig) for y in y_exact]


# Let's plot the data.
plt.plot(x_data, y_data, 'o')
plt.xlabel(r'$x$')
plt.ylabel(r'$y(x)$')
plt.show()




Now that we have some "experimental data", we can try to fit our theoretical model to them.  This means that we want to find the values of $A$, $a$, and $b$ correspond to the model that best goes through the data points.   

In [ ]:
# This is our model.  It takes in the value of the independent variable x, as well
# as the value of the parameters, which are held in the list params.  The function
# returns the prediction for the value of y.
def model(x, params):
    A, a, b = params
    return A*np.exp(-a*x) + b


# Define the objective function, the function that we are trying to minimize.  This
# is the sum of the square of the errors between the predicted value of y and the
# "measured value".
def fobj(params):
    err2 = 0.0
    for x, y in zip(x_data, y_data):
        err = y - model(x, params)
        err2 += err*err
    return err2


params0 = [10, 3, 1]  # This is the initial guess for the optimal value of the parameters

# The function minimize tries to find the value of the parameters that minimize the
# objective function.  The results of the optimization are returned and stored in the
# variable popt.
popt = minimize(fobj, params0)

print(popt)       # Print a summary of the optimization.
print(popt['x'])  # Print the optimal value of the parameters.
A, a, b = popt['x']
print(f'A = {A};  a={a};  b={b}')


# Let's plot
x_fit = np.linspace(0.0, 10.0, 100)
y_fit = [model(x, popt['x']) for x in x_fit]
plt.plot(x_data, y_data, 'o', label='data')
plt.plot(x_fit, y_fit, label='fit')
plt.xlabel(r'$x$')
plt.ylabel(r'$y(x)$')
plt.legend()
plt.show()


If you want, you can try to run this fitting code repeatedly over different sets of "artificially" generated data to see the probability distribution of fitted parameters you will find.

We can also obtain integrals over our spline fit function.  To get the integral between points $a$ and $b$ of the interpolated function, we just type:

`splint(a, b, fit)`

This will return the value of the integral of the function given by `fit` between the limits $a$ and $b$.  

In [ ]:
def Y(x):
    return -0.5*np.exp(-x)*(np.sin(x)+np.cos(x))

x0 = 5

yint_interp = [splint(x0, x, fit) for x in x_interp]
yint_exact = [Y(x) - Y(x0) for x in x_interp]

plt.plot(x_interp, yint_interp, label='cubic interpolation', color='orange')
plt.plot(x_interp, yint_exact, color='blue', ls='dashed', label='exact')
plt.xlabel(r'$x$')
plt.ylabel(r"$\int_{x_0}^x dx'\,y(x')$")
plt.legend()
plt.show()




Finally, the splev function can also be used to find derivatives using the 'der' argument.

In [ ]:
y1_interp = [splev(x, fit, der=1) for x in x_interp]
y1_exact = [np.cos(x)*np.exp(-x)-np.sin(x)*np.exp(-x) for x in x_interp]

plt.plot(x_interp, y1_interp, label='cubic interpolation', color='orange')
plt.plot(x_interp, y1_exact, color='blue', ls='dashed', label='exact')
plt.xlabel(r'$x$')
plt.ylabel(r'$dy/dx$')
plt.legend()
plt.show()

## Fitting adsorption isotherms

In this exercise, we will examine adsorption data for water vapor in zeolite 13X as published by [Zabielska et al. (2020)](https://dx.doi.org/10.24425/cpe.2020.132542).  Figure 3 from the paper is shown below:
![adsorption data](https://github.com/mjksill/CP540-online/blob/main/data/Zabielska_etal_2020/isotherms.png?raw=1)

To begin, digitize the data shown in Fig. 3 of the paper, using [Web Plot Digitizer 4.0](https://apps.automeris.io/wpd4/), to create a CSV file with the adsorption of water $q$ (in units of mol/kg) as a function of partial pressure (in units of Pa) for different temperatures.  The figure can be directly downloaded from Myplace by clicking [here](https://classes.myplace.strath.ac.uk/mod/resource/view.php?id=1786594).



Once you have uploaded the adsorption data, first try to fit the various isotherms with the Langmuir model:
$$
\begin{align*}
\frac{q}{q_{\rm max}}
&= \frac{K p}{1+K p}
\end{align*}
$$
where the parameters of the model are $q_{\rm max}$, the maximum adsorption, and $K$, an adsorption constant related to the free energy of adsorption.  How do the parameteers vary with temperature?

Below is a sketch of some code.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scipy.optimize import curve_fit, minimize

URL = 'https://mjksill.github.io/CP540-online/data/Zabielska_etal_2020/T348.csv'
df = pd.read_csv(URL)
#print(df)


# Langmuir isotherm
def get_q(p, qmax, K):
    q = qmax*K*p / (1.0+K*p)
    return q


qmax = 12.0
K = 1.0e-2
params0 = [qmax, K]
#popt = minimize(fobj, params0)
popt, pcov = curve_fit(get_q, df['p/Pa'], df['q/mol kg-1'], p0=params0)
print(f'best fit parameters: qmax={popt[0]}, K={popt[1]}')


x_list = np.linspace(0, 2500, 100)
y_list = [get_q(p, *popt) for p in x_list]
plt.plot(x_list, y_list)


plt.plot(df['p/Pa'], df['q/mol kg-1'], 'o')

plt.xlim(left=0)
plt.ylim(bottom=0)
plt.xlabel(r'$p / Pa$')
plt.ylabel(r'$q / {\rm mol\,kg^{-1}}$')
plt.show()



Now try to fit the adsorption data with the BET isotherm
$$
\begin{align*}
\frac{q}{q_{\rm max}}
&= \frac{c p}{(1-p_0)(p_0+p(c-1))}
\end{align*}
$$
where the parameters of the model are $q_{\rm max}$, the adsorption of a monolayer, $p_0$, the condensation pressure, and $c$, the BET constant.  How do the parameters vary with temperature?

## Interpolation of the enthalpy of mixtures

The figure below plots the specific enthalpy of mixtures of sulfuric acid and water at various temperatures.  This figure can be downloaded directly from the Myplace site by clicking [here](https://classes.myplace.strath.ac.uk/mod/resource/view.php?id=1786557).

![Enthalpy of mixing](https://github.com/mjksill/CP540-online/blob/main/notebooks/sulfuricacid-water.png?raw=1)


For this exercise, you will need to first digitize the data in the figure, isotherm by isotherm using
[Web Plot Digitizer 4.0](https://apps.automeris.io/wpd4/), and download the data in a CSV file.  Afterward, upload the data into a Pandas dataframe.  Note it will be convenient for you to change the header of the CSV file before you try to upload it using Pandas.

Once you have created a data frame with the digitized data, create a spline fit of each isotherm, using `splrep`


### Questions

1. Plot the variation of the heat capacity with temperature of a 80wt% sulfuric acid solution.  Use SI units.  Note that $1\,{\rm Btu}=1055.06\,{\rm J}$, and $1\,{\rm lb}=0.453592\,{\rm kg}$.

2. One kilogram of water at $0^\circ{\rm C}$ is mixed with one kilogram of an 90wt% sulfuric acid solution at $25^\circ{\rm C}$.  If this is performed adiabatically, what is the final temperature of the mixture?